# Access Metrics Made Easy: Nearest Destination and Access Count
This notebook provides a simple workflow and toolkit to calculate find access metrics. Primarily, this notebook calculates the nearest destination and the number of locations within a given threshold. The basic workflow is as follows:
1. Import data: Point data for destinations, origin geographies, and a transit cost matrix of pre-computed travel costs (eg. Minutes, Miles, etc.)
2. Spatially join destinations to origins: Based on the geospatial location, this associates each destination with an origin geography. Given that the travel cost between each origin geography is known, we can easily calculate the distance between.
3. Calculate metrics: For the nearest location, we'll simply sort the list of origins and destinations by travel time, then take the first entry. For the count within a given threshold, we can filter the list of origins and destinations by the travel time, and count the number of entries under a given threshold.

---

Getting start: Imports and a helper function for later. Here, we'll install the needed libraries (on the Colab remote machine) and import the libraries:

In [1]:
!pip install pandas geopandas access rtree pygeos access pyarrow

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
# from access import Access, weights, Datasets

def dfToGdf(df, lon, lat, crs='EPSG:4326'):
  '''
    df: pandas dataframe
    lon: longitude column name
    lat: latitude column name
    crs: EPSG code or similar coordinate reference system
  '''
  return gpd.GeoDataFrame(
    df.drop([lon, lat], axis=1),
    crs=crs,
    geometry=[Point(xy) for xy in zip(df[lon], df[lat])])

/Users/susan/opt/miniconda3/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Public data:
We've provided a set of public datasets to help get you started. Specifically, we have travel cost matrices for US Zip codes and US Census Tracts, with the travel cost as a value in minutes. The data here is in the Apache Parquet format, an efficient format for storing tabular data.

We've also included some base geographies and population data (should you need it) and some sample destination data of Federally Qualified Health Clinics (FQHC's).

In [3]:
matrices = {
    'tract': {
        'car':'https://uchicago.box.com/shared/static/hkipez75z2p7zivtjdgsfzut4rhm6t6h.parquet',
        'bike':'https://uchicago.box.com/shared/static/cvkq3dytr6rswzrxlgzeejmieq3n5aal.parquet',
        'walk':'https://uchicago.box.com/shared/static/swggh8jxj59c7vpxzx1emt7jnd083rmh.parquet'
    },
    'zip': {
        'car':'https://uchicago.box.com/shared/static/swggh8jxj59c7vpxzx1emt7jnd083rmh.parquet',
        'bike':'https://uchicago.box.com/shared/static/7yzgf1gx3k3sacntjqber6l40m0d5aqw.parquet', 
        'walk':'https://uchicago.box.com/shared/static/lrxeqclmpkflibg9c7sphmun4kny9xsb.parquet',  
    }
}

geographies = {
    'tract': 'https://uchicago.box.com/shared/static/kfoid6fzlbpyfecmwpe9u16cl5n89ru0.zip',
    'zip':'https://uchicago.box.com/shared/static/270ca6syxcg3dlvohhnt3ap92m4t7cxc.zip'
}
pop_data = {
    'tract':'https://uchicago.box.com/shared/static/z6xm6tre935xbc06gg4ukzgyicro26cw.csv',
    'zip': 'https://uchicago.box.com/shared/static/njjpskiuj7amcztrxjws2jfwqlv66t49.csv'
}
sample_point_data = {
    'FQHC': 'https://uchicago.box.com/shared/static/uylcq23g5z8jhvmp7cnofr074j4hwj6e.csv',
    'pharmacies': 'https://uchicago.box.com/shared/static/njjpskiuj7amcztrxjws2jfwqlv66t49.csv',
    'opioid_treatment_facilities': 'https://raw.githubusercontent.com/GeoDaCenter/opioid-policy-scan/master/data_raw/Opioid_Treatment_Directory_Geocoded.csv',
    'hospitals': 'https://raw.githubusercontent.com/GeoDaCenter/opioid-policy-scan/master/data_final/Resources/Hospitals_Geocoded.csv',
    'mentalhealth': 'https://raw.githubusercontent.com/GeoDaCenter/opioid-policy-scan/master/data_final/Resources/MentalHealthProviders_Geocoded.csv',
    'moud_full': 'https://raw.githubusercontent.com/GeoDaCenter/opioid-policy-scan/master/data_final/moud/us-wide-moudsCleaned_geocoded.csv'
}

geoid_cols = {
    "tract":"GEOID",
    "zip": "GEOID10"
}

In [4]:
GEOGRAPHIC_UNIT = 'tract' # 'tract' or 'zip'
TRANSIT_MODE = 'car' # 'car' or 'bike' or 'walk'

TRANSIT_MATRIX = pd.read_parquet(matrices[GEOGRAPHIC_UNIT][TRANSIT_MODE])
GEOGRAPHIES = gpd.read_file(geographies[GEOGRAPHIC_UNIT]).to_crs('EPSG:4326')
GEOGRAPHIES['FIPS'] = GEOGRAPHIES[geoid_cols[GEOGRAPHIC_UNIT]].astype('int64')

Update your destination of interest here (i.e. point locations csv)

In [7]:
DESTINATIONS = dfToGdf(pd.read_csv(sample_point_data['hospitals']), 'Longitude', 'Latitude') 

### Spatial Join
Spatially joining points and polygons is easy. We're using the `intersects` geometric predicate here, simply meaning that if a point intersects a polygon, those two become joined or associated. 

This means that we are able to see which polygon from our geographies each destination is in, and from the travel matrix, we'll know how far it is (roughly) from the other georaphies.

In [8]:
merged_destinations = gpd.sjoin(DESTINATIONS, GEOGRAPHIES[['FIPS', 'geometry']], how='inner', op='intersects')
merged_destinations.head()

/Users/susan/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


,Unnamed: 0,Name,Hospital.Type,Address,Address_2,City,State,Zipcode,County,Staffed.All.Beds,...,Licensed.All.Beds...SOURCE,All.Bed.Occupancy.Rate...SOURCE,ICU.Bed.Occupancy.Rate...SOURCE,CCM_ID,DH.ID,HCRIS.ID,HIFLD.ID,geometry,index_right,FIPS
0,1,IU HEALTH UNIVERSITY HOSPITAL,GENERAL ACUTE CARE,550 UNIVERSITY BLVD,NaN,INDIANAPOLIS,IN,46202,MARION,NaN,...,NaN,None,None,100,NaN,NaN,100,POINT (-86.17656 39.77528),31285,18097391000
79,80,RILEY HOSPITAL FOR CHILDREN,CHILDREN,705 RILEY HOSPITAL DR,NaN,INDIANAPOLIS,IN,46202,MARION,NaN,...,NaN,None,None,102,NaN,NaN,102,POINT (-86.18028 39.77711),31285,18097391000
1818,1819,ESKENAZI HEALTH,GENERAL ACUTE CARE,720 ESKENAZI AVENUE,NaN,INDIANAPOLIS,IN,46254,MARION,316.0,...,DH-NUM_LICENS,HCRIS-Total Bed Occupancy Rate,HCRIS-ICU Occupancy Rate,157146254,1267.0,150024.0,157146254,POINT (-86.18399 39.77788),31285,18097391000
4669,4670,RICHARD L. ROUDEBUSH VA MEDICAL CENTER,MILITARY,1481 W 10TH ST,NaN,INDIANAPOLIS,IN,46202,MARION,NaN,...,NaN,None,None,4646202,1268.0,NaN,4646202,POINT (-86.18716 39.77846),31285,18097391000
1,2,HEALTHSOUTH NEW ENGLAND REHABILIATION AT BEVERLY,REHABILITATION,"800 CUMMINGS CENTER, 1ST FL, UNIT147",NaN,BEVERLY,MA,1915,ESSEX,NaN,...,HIFLD-BEDS,None,None,10001915,NaN,NaN,10001915,POINT (-70.88724 42.56099),24873,25009217300


### Moins Est Plus
Less is more, let's just snag the columns we need. We'll need to join this data again to the travel matrix, so the second line gets everyone speaking the same language.

In [9]:
## Pull out the simplified columns we need for the analysis
destinations_simplified = merged_destinations[['index_right','FIPS']]
destinations_simplified.head()
#destinations_simplified.shape

,index_right,FIPS
0,31285,18097391000
79,31285,18097391000
1818,31285,18097391000
4669,31285,18097391000
1,24873,25009217300


### The Big Join
Currently, we have destinations associated with our origin geographies (if you're using the default data, census tracts and health clinics), but we need to bring it all together by joining the origins and destinations to the travel matrix. Below, we'll join the travel matrix to the destinations:

In [10]:
## Merge onto the transit matrix, giving us the distance from each origin to each destination
merge_transit_matrix = TRANSIT_MATRIX.merge(destinations_simplified, left_on="destination", right_on="FIPS")
merge_transit_matrix.head()

,origin,destination,minutes,index_right,FIPS
0,17129010200,17117956400,94.12,27085,17117956400
1,17129010300,17117956400,102.03,27085,17117956400
2,17129010100,17117956400,88.56,27085,17117956400
3,17115003100,17117956400,105.26,27085,17117956400
4,17115002602,17117956400,108.48,27085,17117956400


### Analysis Time
Let's get down to business. To begin, let's declare some variables that will help us a bit later. To start, we can define what our origin column (by default, creatively, `origin`), the destination ID column that came from the destinations data, the travel cost column, and the treshold for travel time. 

In [11]:
origin_col = 'origin'
destination_id_col = 'index_right'
travel_cost_col = 'minutes'
travel_threshold = 30

### Data Cleanup
We have some weird -1000 values. Let's fix them and replace them with 999, the default null value of this travel matrix.

In [12]:
## clean up this weird bug, then merge the data
travel_costs = merge_transit_matrix.sort_values(travel_cost_col, ascending=True)
travel_costs.minutes = travel_costs.minutes.replace(-1000, 999)
travel_costs.origin = travel_costs.origin.astype('int64')
travel_costs.head()

,origin,destination,minutes,index_right,FIPS
6314187,15005031900,15003003502,999.0,64180,15003003502
6317965,15009031405,15009031700,999.0,40992,15009031700
6317964,15009990200,15009031700,999.0,40992,15009031700
6317961,15003011105,15009031700,999.0,40992,15009031700
6317164,15005990000,15003006701,999.0,70576,15003006701


### Nearest location
To get the nearest location, sort the values by lowest cost then filter for  the first appearance of each origin ID. This means we'll get the first time that origin shows up, sorted by the lowest travel cost. 

We'll use pandas `.duplicated()` function with the not (`~`) operator before it.

In [13]:
time_to_nearest = travel_costs[~travel_costs.origin.duplicated()][[origin_col, travel_cost_col]]
time_to_nearest.head()

,origin,minutes
6314187,15005031900,999.0
6317965,15009031405,999.0
6317964,15009990200,999.0
6317961,15003011105,999.0
6317164,15005990000,999.0


### Count in Threshold 
For getting the count of destinations within a given threshold (by default, 30 minutes), we can chain a couple functions here from pandas. 

First, we'll filter the `travel_costs` dataframe for costs that are less than or equal to the threshold. Then group by the origin column, giving us sets of rows that share the same origin ID, and then count those columns, giving us the number of rows for each origin ID with a travel cost under our treshold.

We'll re-label some columns for easy reference.

In [14]:
## For count, we simple filter for the cost under a given threshold
## Then group by and count the results
count_within_threshold = travel_costs[travel_costs[travel_cost_col] <= travel_threshold] \
  .groupby(origin_col).count() \
  .reset_index()[[origin_col, travel_cost_col]]
count_within_threshold.columns = [origin_col, f"count within {travel_threshold}"]

### Merge Results
Now, we can merge our two findings into an easy, breezy, beautiful dataframe.

In [15]:
merged_metrics = count_within_threshold.merge(time_to_nearest, on=origin_col, how="outer")
merged_metrics.head()

,origin,count within 30,minutes
0,1001020100,3.0,11.32
1,1001020200,3.0,7.42
2,1001020300,4.0,0.00
3,1001020400,4.0,5.70
4,1001020500,7.0,7.50


### Cleanup
One last edge case to handle here: It is possible some origins are not within 30 minutes of a destination, meaning some of the data will be null. Or, we might have lost some origins from the full geographies dataset.

While not the end of the world, we can clean this up here before shipping of results to our (soon to be disgruntled) data scientist colleagues. 

The below finds the missing origin IDs and fills them in, giving us the revered `findings` dataframe. 

🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉

In [16]:
## To clean up any missing data, we can check back with our origin list
analyzed_origins = list(merged_metrics[origin_col])
missing_origins = [o for o in GEOGRAPHIES.FIPS if o not in analyzed_origins]

## Then, fill the missing data
missing_data = []
for o in missing_origins:
    missing_entry = {}
    missing_entry[origin_col] = o
    missing_entry[f"count within {travel_threshold}"]=0
    missing_entry[travel_cost_col]=None
    missing_data.append(missing_entry)
missing_df = pd.DataFrame(missing_data)

## and concatenate results
findings = pd.concat([merged_metrics, missing_df])
# Fill any null values with 0 for count within
findings['count within 30'] = findings['count within 30'].fillna(0).astype(int)
# Replace error value "999" in matrices with blanks
findings['minutes'] = findings['minutes'].replace(999.0, None)
findings.head()

,origin,count within 30,minutes
0,1001020100,3,11.32
1,1001020200,3,7.42
2,1001020300,4,0.00
3,1001020400,4,5.70
4,1001020500,7,7.50


### What's Next?
Well, now you could take this data and export it as a CSV, or join it back to the geographies and visualize it, or try running this analysis with some different data. Ball is in your court, you got this!


In [17]:
# Export to csv
findings.to_csv('hospitals_drive_tract.csv', index = False)